# Importing Library

In [4]:
import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

from tfx.proto import trainer_pb2

# Declaring Contants Variable

Code di bawah akan membuat variable konstan untuk pipeline

In [5]:
PIPELINE_NAME = "diabetes-pipeline"
SCHEMA_PIPELINE_NAME = "diabetes-tfdv-schema"

PIPELINE_ROOT = os.path.join('komarr007-pipeline', PIPELINE_NAME)

METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [6]:
DATA_ROOT = "data"

In [7]:
ctx = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Example Gen

code di bawah melakukan pembacaan data

In [8]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)
ctx.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 83
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# Statistic Gen

Code di bawah membuat statistic generator.

In [9]:
stats_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

ctx.run(stats_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 84
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [10]:
ctx.show(stats_gen.outputs['statistics'])

# Schema Gen

Cde di bawah melakukan kontrol terhadap skema pada data

In [11]:
schema_gen = SchemaGen(statistics=stats_gen.outputs['statistics'])

ctx.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 85
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [12]:
ctx.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'BMI',FLOAT,required,,-
'BloodPressure',INT,required,,-
'DiabetesPedigreeFunction',FLOAT,required,,-
'Glucose',INT,required,,-
'Insulin',INT,required,,-
'Outcome',INT,required,,-
'Pregnancies',INT,required,,-
'SkinThickness',INT,required,,-


# Anomalies Validator

Code di bawah akan mengecek anomali pada data

In [13]:
example_validator = ExampleValidator(
    statistics=stats_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
ctx.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 86
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [14]:
ctx.show(example_validator.outputs['anomalies'])

# Create Constant Module

code di bawah akan membuat contant module yang akan di gunakan untuk transformasi data

In [15]:
CONSTANT_MODULE_FILE = "diabetes_data_constant.py"

In [16]:
%%writefile {CONSTANT_MODULE_FILE}

import numpy as np
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "Outcome"
FEATURE_KEY = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']


def transformed_name(key):
    """Rename transformed features"""

    return key + "_tn"

Overwriting diabetes_data_constant.py


# Create Transform Module

Code di bawah membuat transformasi module

In [17]:
TRANSFORM_MODULE_FILE = "diabetes_data_transform.py"

In [18]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
import diabetes_data_constant

_FEATURE_KEY = diabetes_data_constant.FEATURE_KEY
_LABEL_KEY = diabetes_data_constant.LABEL_KEY
_transformed_name = diabetes_data_constant.transformed_name

def preprocessing_fn(inputs):
  """Melakukan proses scaling z score pada feature"""
  
  outputs = {}
  for key in _FEATURE_KEY:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])

  outputs[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]

  return outputs

Overwriting diabetes_data_transform.py


# Transform pipeline

Code di bawah membuat transformasi untuk pipeline berdasarkan module transform yang telah dibuat

In [19]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
ctx.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: komarr007-pipeline\diabetes-pipeline\Transform\transform_graph\87\.temp_path\tftransform_tmp\7500dfceb1a5450d81cd04c25e4a2ae1\assets


INFO:tensorflow:Assets written to: komarr007-pipeline\diabetes-pipeline\Transform\transform_graph\87\.temp_path\tftransform_tmp\7500dfceb1a5450d81cd04c25e4a2ae1\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 87
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# Create Training Module

Code di bawah membuat training module yang berisikan model ML

In [17]:
TRAINER_MODULE_FILE = "diabetes_data_trainer.py"

In [18]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
from tensorflow import keras
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
import diabetes_data_constant

_LABEL_KEY = diabetes_data_constant.LABEL_KEY
_FEATURE_KEY = diabetes_data_constant.FEATURE_KEY
_transformed_name = diabetes_data_constant.transformed_name

    
def get_model(show_summary: bool = True) -> tf.keras.models.Model:

    input_features = []

    for key in _FEATURE_KEY:
        input_features.append(
            tf.keras.Input(shape=(1,), name=_transformed_name(key))
        )

    x = tf.keras.layers.concatenate(input_features)
    x_1 = tf.keras.layers.Dense(16, activation="relu")(x)

    output = tf.keras.layers.Dense(1, activation="sigmoid")(x_1)

    inputs = input_features

    keras_model = tf.keras.models.Model(inputs, output)
    keras_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(),
        ],
    )
    if show_summary:
        keras_model.summary()

    return keras_model


def _gzip_reader_fn(filenames):
    """membaca gzip file dari input"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """fungsi untuk parsing tf.Example."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """output untuk digunakan pada saat serving"""
        
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        outputs = model(transformed_features)
        return {"outputs": outputs}

    return serve_tf_examples_fn


def _input_fn(file_pattern, tf_transform_output, batch_size=64):

    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=_transformed_name(_LABEL_KEY),
    )

    return dataset


def run_fn(fn_args):
    """main function"""
    
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 64)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 64)

    print(train_dataset)
    
    model = get_model()

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )
    callbacks = [tensorboard_callback]

    model.fit(
        train_dataset,
        epochs=10,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
    )

    signatures = {
        "serving_default": _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

Overwriting diabetes_data_trainer.py


# Model Pipeline

Code di bawah akan mengeksekusi model pada module yang telah dibuat

In [19]:
from tfx.proto import trainer_pb2
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

trainer = Trainer(
    module_file = os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(num_steps=100),
    eval_args = trainer_pb2.EvalArgs(num_steps=1)
)

ctx.run(trainer)

<PrefetchDataset element_spec=({'Age_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'BMI_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'BloodPressure_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'DiabetesPedigreeFunction_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'Glucose_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'Insulin_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'Pregnancies_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None), 'SkinThickness_tn': TensorSpec(shape=(64, 1), dtype=tf.float32, name=None)}, TensorSpec(shape=(64, 1), dtype=tf.int64, name=None))>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Pregnancies_tn (InputLayer)    [(None, 1)]          0           []                               
               

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: komarr007-pipeline\diabetes-pipeline\Trainer\model\77\Format-Serving\assets


INFO:tensorflow:Assets written to: komarr007-pipeline\diabetes-pipeline\Trainer\model\77\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 77
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

# Resolver

Code di bawah akan melakukan resolver

In [20]:
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
ctx.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 78
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

# Evaluator

Code di bawah akan mengevaluasi model yang telah dibuat

In [23]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Outcome')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [24]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
ctx.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 80
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [25]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Pusher

Code dibawah membuat model siap untuk di serving

In [28]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/diabetes-detection-model'))
)
 
ctx.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 82
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [2]:
MODEL_PATH = "./serving_model_dir/diabetes-detection-model/1676195266"
model = tf.keras.models.load_model(MODEL_PATH)

In [20]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)

features {
  feature {
    key: "Age_tn"
    value {
      float_list {
        value: 1.4023081064224243
      }
    }
  }
  feature {
    key: "BMI_tn"
    value {
      float_list {
        value: 0.1848072111606598
      }
    }
  }
  feature {
    key: "BloodPressure_tn"
    value {
      float_list {
        value: 0.15202367305755615
      }
    }
  }
  feature {
    key: "DiabetesPedigreeFunction_tn"
    value {
      float_list {
        value: 0.4571339786052704
      }
    }
  }
  feature {
    key: "Glucose_tn"
    value {
      float_list {
        value: 0.8456256985664368
      }
    }
  }
  feature {
    key: "Insulin_tn"
    value {
      float_list {
        value: -0.703625500202179
      }
    }
  }
  feature {
    key: "Outcome_tn"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "Pregnancies_tn"
    value {
      float_list {
        value: 0.6715670228004456
      }
    }
  }
  feature {
    key: "SkinThickness_tn"
    value 

In [56]:
import numpy as np
import json
import requests

# Define the input data
test_data = [(1.4023081064224243, 0.1848072111606598, 0.15202367305755615, 0.4571339786052704, 0.8456256985664368,
 -0.703625500202179, 0.6715670228004456, 0.8614551424980164)]

# Convert the input data into a list of arrays
# inputs = np.array([[x] for x in test_data[0]]).reshape(-1)
flattened_test_data = [value for example in test_data for value in example]

print(flattened_test_data)

inputs = [str(x) for x in flattened_test_data]

headers = {"content-type": "application/json"}

json_data = json.dumps({"signature_name": "","instances": inputs})


endpoint = "http://localhost:2020/v1/models/diabetes-detection-model:predict"


response = requests.post(endpoint, data=json_data)
prediction_result = response.json()
print(prediction_result)

[1.4023081064224243, 0.1848072111606598, 0.15202367305755615, 0.4571339786052704, 0.8456256985664368, -0.703625500202179, 0.6715670228004456, 0.8614551424980164]
{'error': "Could not parse example input, value: '1.4023081064224243'\n\t [[{{function_node __inference_serve_tf_examples_fn_13191}}{{node ParseExample/ParseExampleV2}}]]"}


In [22]:
import requests
import pprint
 
pp = PrettyPrinter()
print(requests.get("http://localhost:2020/v1/models/diabetes-detection-model/metadata").json())

{'model_spec': {'name': 'diabetes-detection-model', 'signature_name': '', 'version': '1676195266'}, 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'examples': {'dtype': 'DT_STRING', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_examples:0'}}, 'outputs': {'outputs': {'dtype': 'DT_FLOAT', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '1', 'name': ''}], 'unknown_rank': False}, 'name': 'StatefulPartitionedCall:0'}}, 'method_name': 'tensorflow/serving/predict'}, '__saved_model_init_op': {'inputs': {}, 'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID', 'tensor_shape': {'dim': [], 'unknown_rank': True}, 'name': 'NoOp'}}, 'method_name': ''}}}}}
